In [181]:
import os
import os.path as op
import datetime
import multiprocessing
import re
import shutil
import subprocess
import sys
import time
from concurrent.futures import ThreadPoolExecutor
from glob import glob
from importlib import reload

import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

sys.path.append(op.join(op.dirname(os.getcwd()), "qc"))
sys.path.append(op.join(op.dirname(os.getcwd()), "setup"))
sys.path.append(op.join(op.dirname(os.getcwd()), "utils"))
import utilities as uts

sys.path.append("/home/mac/dschonhaut/code")
from general.basic import helper_funcs as hf

sys.path.append(op.join(op.expanduser("~"), "code", "style"))
from colors import get_colors

co, palettes = get_colors()

from general.basic.config import get_plot_defaults, set_rcparams

mpl.rcParams = set_rcparams(mpl.rcParams)
mpl.rcParams["axes.grid"] = False
d = get_plot_defaults()
co = d.get("colors", None)
colws = d.get("colws", None)
font = d.get("font", None)
lws = d.get("lws", None)
pad = d.get("pad", None)
palettes = d.get("palettes", None)

In [29]:
new_sys_path = []
for d in sys.path:
    if d and d not in new_sys_path:
        new_sys_path.append(d)
sys.path = new_sys_path

In [150]:
reload(create_qc_evals)

<module 'create_qc_evals' from '/mnt/coredata/processing/leads/code/qc/create_qc_evals.py'>

In [72]:
# sys.path.append(op.join(op.dirname(os.getcwd()), "qc"))
import create_qc_evals

In [5]:
qc_dir = "/mnt/coredata/processing/leads/metadata/qc"
qc_evals = {
    "MRI-T1": pd.read_csv(
        uts.glob_sort(op.join(qc_dir, "processed_MRI-T1_qc-evals_*.csv"))[-1]
    ),
    "FBB": pd.read_csv(
        uts.glob_sort(op.join(qc_dir, "processed_FBB_qc-evals_*.csv"))[-1]
    ),
    "FDG": pd.read_csv(
        uts.glob_sort(op.join(qc_dir, "processed_FDG_qc-evals_*.csv"))[-1]
    ),
    "FTP": pd.read_csv(
        uts.glob_sort(op.join(qc_dir, "processed_FTP_qc-evals_*.csv"))[-1]
    ),
}

for k, v in qc_evals.items():
    print(f"{k}: {v.shape}")

MRI-T1: (1155, 10)
FBB: (997, 10)
FDG: (158, 9)
FTP: (959, 10)


In [108]:
proc_dir = "/mnt/coredata/processing/leads/data/processed"
scan_types = ["MRI-T1", "FBB", "FDG", "FTP"]
timestamp = "2024-07-02-00-00-00"
overwrite = True
for scan_type in qc_evals:
    for idx, row in qc_evals[scan_type].iterrows():
        # Get the scan directory
        scan_dir = op.join(proc_dir, row["subj"], f"{scan_type}_{row['scan_date']}")

        # Make sure the scan directory exists
        if not op.isdir(scan_dir):
            raise FileNotFoundError(scan_dir)

        # Create the scan QC directory
        scan_qc_dir = op.join(scan_dir, "qc")
        os.makedirs(scan_qc_dir, exist_ok=True)

        # Save the QC eval
        scan_tag = uts.get_scan_tag(scan_dir)
        qc_eval_file = op.join(scan_qc_dir, f"{scan_tag}_qc-eval_{timestamp}.csv")
        qc_eval = pd.DataFrame(row).reset_index()
        qc_eval.columns = ["field", "value"]
        if overwrite or not op.isfile(qc_eval_file):
            qc_eval.to_csv(qc_eval_file, index=False)
            # print(f"Saved {qc_eval_file}")

In [107]:
pd.DataFrame(row).reset_index()

,index,958
0,subj,LDS9410679
1,scan_date,2023-10-11
2,rater,Stefania
3,native_pet_ok,1
4,pet_to_mri_coreg_ok,1
5,infcblgm_mask_ok,1
6,erodedwm_mask_ok,1
7,affine_pet_ok,1
8,warped_pet_ok,1
9,notes,NaN


In [58]:
subj = row["subj"]
scan_date = row["scan_date"]

In [82]:
columns = create_qc_evals.get_qc_eval_fields(scan_type)
[subj, scan_date] + [np.nan] * (len(columns) - 2)
pd.Series(
    index=pd.Index(columns, name="field"),
    data=[subj, scan_date] + [np.nan] * (len(columns) - 2),
    name="value",
).reset_index()
# pd.DataFrame(data=[subj, scan_date] + [np.nan] * (len(columns) - 2), columns=columns)

,field,value
0,subj,LDS9410679
1,scan_date,2023-10-11
2,rater,NaN
3,native_pet_ok,NaN
4,pet_to_mri_coreg_ok,NaN
5,infcblgm_mask_ok,NaN
6,erodedwm_mask_ok,NaN
7,affine_pet_ok,NaN
8,warped_pet_ok,NaN
9,notes,NaN


In [176]:
pd.concat(
    [
        pd.read_csv(qc_eval_file).set_index("field").T,
        pd.read_csv(qc_eval_file).set_index("field").T,
    ],
    ignore_index=True,
)

field,subj,scan_date,rater,native_pet_ok,pet_to_mri_coreg_ok,infcblgm_mask_ok,erodedwm_mask_ok,affine_pet_ok,warped_pet_ok,notes
0,LDS9410679,2023-10-11,Stefania,1,1,1,1,1,1,NaN
1,LDS9410679,2023-10-11,Stefania,1,1,1,1,1,1,NaN


In [90]:
qc_eval_file = create_qc_evals.create_qc_eval_file(scan_dir)
print(qc_eval_file)

/mnt/coredata/processing/leads/data/processed/LDS9410679/FTP_2023-10-11
Saved /mnt/coredata/processing/leads/data/processed/LDS9410679/FTP_2023-10-11/qc/LDS9410679_FTP_2023-10-11_qc-eval_2024-08-15-11-25-55.csv


In [146]:
qc_eval_file = create_qc_evals.get_qc_eval_file(scan_dir)
print(qc_eval_file)

/mnt/coredata/processing/leads/data/processed/LDS9410679/FTP_2023-10-11/qc/LDS9410679_FTP_2023-10-11_qc-eval_2024-07-02-00-00-00.csv


In [122]:
qc_eval = pd.read_csv(qc_eval_file).set_index("field")["value"]
for key, val in qc_eval.items():
    print(f"{key}: {val}")

subj: LDS9410679
scan_date: 2023-10-11
rater: Stefania
native_pet_ok: 1
pet_to_mri_coreg_ok: 1
infcblgm_mask_ok: 1
erodedwm_mask_ok: 1
affine_pet_ok: 1
warped_pet_ok: 1
notes: nan


In [156]:
reload(create_qc_evals)

<module 'create_qc_evals' from '/mnt/coredata/processing/leads/code/qc/create_qc_evals.py'>

In [177]:
proj_dir = "/mnt/coredata/processing/leads"
qc_evals = create_qc_evals.merge_completed_qc_eval_files(proj_dir)

Saved /mnt/coredata/processing/leads/metadata/qc/processed_MRI-T1_qc-evals_2024-08-15-12-14-29.csv
Saved /mnt/coredata/processing/leads/metadata/qc/processed_FBB_qc-evals_2024-08-15-12-14-29.csv
Saved /mnt/coredata/processing/leads/metadata/qc/processed_FDG_qc-evals_2024-08-15-12-14-29.csv
Saved /mnt/coredata/processing/leads/metadata/qc/processed_FTP_qc-evals_2024-08-15-12-14-29.csv


In [187]:
for scan_type in incomplete_qc_scans:
    for scan_dir in incomplete_qc_scans[scan_type]:
        scan_qc_dir = op.join(scan_dir, "qc")
        if not op.isdir(scan_qc_dir):
            cmd = f"/mnt/coredata/processing/leads/code/qc/create_qc_evals.py create -s {scan_dir}"
            subprocess.run(cmd, shell=True)
        else:
            continue

Traceback (most recent call last):
  File "/mnt/coredata/processing/leads/code/qc/create_qc_evals.py", line 360, in <module>
    create_qc_eval_file(scan_dir)
  File "/mnt/coredata/processing/leads/code/qc/create_qc_evals.py", line 101, in create_qc_eval_file
    os.makedirs(qc_dir)
  File "<frozen os>", line 225, in makedirs
PermissionError: [Errno 13] Permission denied: '/mnt/coredata/processing/leads/data/processed/LDS0370646/MRI-T1_2024-08-12/qc'
Traceback (most recent call last):
  File "/mnt/coredata/processing/leads/code/qc/create_qc_evals.py", line 360, in <module>
    create_qc_eval_file(scan_dir)
  File "/mnt/coredata/processing/leads/code/qc/create_qc_evals.py", line 101, in create_qc_eval_file
    os.makedirs(qc_dir)
  File "<frozen os>", line 225, in makedirs
PermissionError: [Errno 13] Permission denied: '/mnt/coredata/processing/leads/data/processed/LDS3600350/FBB_2024-08-06/qc'
Traceback (most recent call last):
  File "/mnt/coredata/processing/leads/code/qc/create_qc_e

In [188]:
incomplete_qc_scans = create_qc_evals.find_scans_with_incomplete_qc(proj_dir)

Processing complete and QC needed for the following 31 MRI-T1 scans:
  /mnt/coredata/processing/leads/data/processed/LDS0100330/MRI-T1_2024-06-07
  /mnt/coredata/processing/leads/data/processed/LDS0360453/MRI-T1_2024-06-13
  /mnt/coredata/processing/leads/data/processed/LDS0370495/MRI-T1_2024-07-23
  /mnt/coredata/processing/leads/data/processed/LDS0730558/MRI-T1_2024-01-25
  /mnt/coredata/processing/leads/data/processed/LDS0220522/MRI-T1_2024-02-07
  /mnt/coredata/processing/leads/data/processed/LDS9410608/MRI-T1_2024-02-09
  /mnt/coredata/processing/leads/data/processed/LDS0370304/MRI-T1_2024-06-10
  /mnt/coredata/processing/leads/data/processed/LDS1770634/MRI-T1_2024-07-10
  /mnt/coredata/processing/leads/data/processed/LDS0730628/MRI-T1_2024-07-25
  /mnt/coredata/processing/leads/data/processed/LDS0990407/MRI-T1_2024-07-18
  /mnt/coredata/processing/leads/data/processed/LDS0730625/MRI-T1_2024-07-08
  /mnt/coredata/processing/leads/data/processed/LDS0350614/MRI-T1_2024-06-24
  /mnt/

In [200]:
idx = np.argsort([op.basename(d).split("_")[1] for d in incomplete_qc_scans["FBB"]])
np.array(incomplete_qc_scans["FBB"])[idx].tolist()

['/mnt/coredata/processing/leads/data/processed/LDS0370451/FBB_2024-03-27',
 '/mnt/coredata/processing/leads/data/processed/LDS0180605/FBB_2024-05-29',
 '/mnt/coredata/processing/leads/data/processed/LDS1770688/FBB_2024-05-31',
 '/mnt/coredata/processing/leads/data/processed/LDS9410367/FBB_2024-06-05',
 '/mnt/coredata/processing/leads/data/processed/LDS0100330/FBB_2024-06-06',
 '/mnt/coredata/processing/leads/data/processed/LDS1770580/FBB_2024-06-07',
 '/mnt/coredata/processing/leads/data/processed/LDS1770686/FBB_2024-06-11',
 '/mnt/coredata/processing/leads/data/processed/LDS3600602/FBB_2024-06-11',
 '/mnt/coredata/processing/leads/data/processed/LDS0370609/FBB_2024-06-13',
 '/mnt/coredata/processing/leads/data/processed/LDS0360453/FBB_2024-06-13',
 '/mnt/coredata/processing/leads/data/processed/LDS0220502/FBB_2024-06-13',
 '/mnt/coredata/processing/leads/data/processed/LDS0360584/FBB_2024-06-18',
 '/mnt/coredata/processing/leads/data/processed/LDS0100352/FBB_2024-06-20',
 '/mnt/cored